In [ ]:
import pandas as pd
import matplotlib
import plotly.express as px
import matplotlib.pyplot as plt
from collections import namedtuple

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_deg.xlsx')
df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_degradated_20to40_component_only.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_degradated_20to40_all_level.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_degradated_15to35_component_only.xlsx')
df.shape

In [ ]:
df = thermo_df(df)

In [ ]:
df.to_excel('/Users/bryan/Downloads/ori.xlsx')

In [ ]:
plt = plot_zone(df[df.Mass < 20000])

plt.savefig('/Users/bryan/Downloads/tRNA_full_dots.png', transparent=True)

In [ ]:
df = df.sort_values('Vol', ascending=False)

In [ ]:
# df_top = df[(10000 < df.Mass) & (df.Mass  < 20000)].head(100)
# df_top = df[(df.RT > 2.0) & (df.Mass < 20000)].head(1000)
df_top = local_top(df, top=2)
plotly_zone(df_top)

In [ ]:
# df_top = df[(df.Mass > 4000) & (df.Mass < 20000) & (df.RT > 2.2)].sort_values('Vol').groupby('RT').tail(3)
df_top = df[df.Mass < 20000].sort_values('Vol').groupby('RT').tail(10)
# df_top = df[(df.RT > 2.0) & (df.Mass < 20000)].sort_values('Vol').groupby('RT').tail(2)
plot_zone(df_top)
df_top.shape

In [ ]:
df_top.to_excel('/Users/bryan/Downloads/bak.xlsx')

In [ ]:
import plotly.graph_objects as go

idxs = list()
# create our callback function
def update_point(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

# df_draw = df2[(df2.Mass > 12329) & (df2.Mass < 22000) & (df2.RT < 25) & (df2.RT > 8)]
df_draw = df_top #df7
# df_draw = df2_top[(df2_top.RT < 25) & (df2_top.RT > 8)]
x = df_draw.Mass
y = df_draw.RT
fig = go.Scatter(x=x, y=y, mode='markers')
f = go.FigureWidget(fig)
f.update_layout(autosize=False, width=1024, height=700, paper_bgcolor="LightSteelBlue",
    margin=dict(
        l=20,
        r=20,
        b=20,
        t=20,
        pad=4
    ))
scatter = f.data[0]
scatter.on_selection(update_point)
f

In [ ]:
df_bak = df_chosen

In [ ]:
import plotly.graph_objects as go

idxs = list()
# create our callback function
def update_point(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

# df_draw = df2[(df2.Mass > 12329) & (df2.Mass < 22000) & (df2.RT < 25) & (df2.RT > 8)]
df_draw = bcr_5p[0]#df_bak[df_bak.Mass < 12000] #df7
# df_draw = df2_top[(df2_top.RT < 25) & (df2_top.RT > 8)]
x = df_draw.Mass
y = df_draw.RT
fig = go.Scatter(x=x, y=y, mode='markers')
f = go.FigureWidget(fig)
f.update_layout(autosize=False, width=1024, height=700, paper_bgcolor="LightSteelBlue",
    margin=dict(
        l=20,
        r=20,
        b=20,
        t=20,
        pad=4
    ))
scatter = f.data[0]
scatter.on_selection(update_point)
f

In [ ]:
df_chosen = df_draw.iloc[idxs]
df_chosen.info()

In [ ]:
bcr_5p = base_calling_random(df[df.Mass < 20000])
plt, _ = plot_basecalling(*bcr_5p, annotate=False)
# plt.savefig('/Users/bryan/Downloads/tRNA_basecalling.png', transparent=True)

In [ ]:
full_mass = 23904.236#23057.09
print(full_mass)
df_ms, _ = computational_data_seperation(df, df, full_mass=full_mass, ignore_endpoints=True)
df_ms.shape
plot_zone(df_ms)

# Processing ~25k Dalton

In [ ]:
df_25k = df[(df.Mass > 2.1E4) & (df.Mass < 3E4)]
df_25k.shape

In [ ]:
# df_25k.to_excel('~/Downloads/total_tRNA_20to40_ad_25k.xlsx')
df_25k.to_excel('~/Downloads/total_tRNA_sample4_ad_25k.xlsx')

In [ ]:
# plot_zones(df, df_25k)
plt.figure(figsize=(12, 9))
plt.xlabel('Mass')
plt.ylabel('RT')
plt.scatter(df.Mass, df.RT)
plt.scatter(df_25k.Mass, df_25k.RT, color='g')


In [ ]:
# plot_zone(df_25k)

plt.figure(figsize=(12, 9))
plt.xlabel('Mass')
plt.ylabel('RT')
plt.scatter(df_25k.Mass, df_25k.RT, color='g')

# plt.savefig('/Users/bryan/Downloads/tRNA_25k.png')

In [ ]:
df_25k.sort_values('Mass', ascending=False)

# Sliding Mass Window (600Da)

In [ ]:
def process_slopes(df, mass_pairs, radius=0.25):
    slopes = list()
    for t in mass_pairs:
        df_pair = df[df.Mass.isin(t)]
        if df_pair.empty:
            continue
        
        slope = 1E3 * (df_pair.iloc[0].RT - df_pair.iloc[1].RT)/(df_pair.iloc[0].Mass - df_pair.iloc[1].Mass)
        if slope > 0.:
            slopes.append(slope)
#     median = np.median(slopes)
#     lo = median - radius
#     hi = median + radius
#     print(slopes)
#     median = slope_range(slopes, radius * 2)
#     if not median:
#         return None
#     lo = median.lo
#     hi = median.hi
    ranges = slope_ranges(slopes)
        
    count = 0
    count_ignore = 0
    accepted_pairs = list()
    for t in mass_pairs:
        df_pair = df[df.Mass.isin(t)]
        if df_pair.empty:
            continue
        
        slope = 1E3 * (df_pair.iloc[0].RT - df_pair.iloc[1].RT)/(df_pair.iloc[0].Mass - df_pair.iloc[1].Mass)
#         if slope > 0.:
#         if lo < slope < hi:
        if any([i[0] < slope < i[1] for i in ranges]):
            slopes.append(slope)
#             plt.plot(df_pair.Mass, df_pair.RT, 'green')
            count += 1
            accepted_pairs.append(df_pair)
        else:
            count_ignore += 1
    
#     print('median {:.2f} lo-hi {:.3f}-{:.3f} accept {} reject {}'.format((lo+hi)/2, lo, hi, count, count_ignore))
    print('accept {} reject {}'.format(count, count_ignore))
    return accepted_pairs

In [ ]:
def slope_range(l, win=0.2):
    l = np.array(l)
    l.sort()
    len = l.shape[0]
    WinCount = namedtuple('WinCount', 'lo hi count')
    win_counts = list()
    for value in l:
        lo = value
        hi = lo + win
        count = l[(l>=lo) & (l < hi)].shape[0]
        win_count = WinCount(lo, hi, count)
        win_counts.append(win_count)
    
    if win_counts:
        max_win_count = max(win_counts, key=lambda x: x.count)
        return max_win_count

In [ ]:
def slope_ranges(l, ratio=0.5, top=3):
    hist = np.histogram(l, bins='auto')
    max_bin_count = max(hist[0])
    loc_counts = list(enumerate(hist[0]))
    top_loc_counts = [i for i in loc_counts if i[1] > max_bin_count * ratio]
    top_loc_counts = sorted(top_loc_counts, key=lambda x: x[1], reverse=True)[:top]
    ranges = [(hist[1][i[0]], hist[1][i[0]+1]) for i in top_loc_counts]
    print(ranges)
    return ranges

In [ ]:
def plot_basecalling2(df, mass_pairs, endpoints=pd.DataFrame(), annotate=True, plt=None, median=-1):
    if not plt:
        plt = matplotlib.pyplot
    fig = plt.figure(figsize=(16, 12))
    plt.xlabel('Monoisotopic Mass (Da)', fontname="Arial", fontsize=15, color='black')
    plt.ylabel('Retention Time (min)', fontname="Arial", fontsize=15, color='black')
    plt.xticks(fontname="Arial", size=13, color='black')
    plt.yticks(fontname="Arial", size=13, color='black')
        
    plt.scatter(df.Mass, df.RT)

    # calc slopes
    slopes = list()
    for t in mass_pairs:
        df_pair = df[df.Mass.isin(t)]
        if df_pair.empty:
            continue
        
        slope = 1E3 * (df_pair.iloc[0].RT - df_pair.iloc[1].RT)/(df_pair.iloc[0].Mass - df_pair.iloc[1].Mass)
        if slope > 0.:
            slopes.append(slope)
#     if median < 0:
#         median = np.median(slopes)
#     lo = median - 0.25
#     lo = lo if lo > 0 else 0
#     hi = median + 0.25
#     median = slope_range(slopes, 0.5)
#     if not median:
#         return plt, fig
#     lo = median.lo
#     hi = median.hi
    ranges = slope_ranges(slopes)
        
    count = 0
    count_ignore = 0
    for t in mass_pairs:
        df_pair = df[df.Mass.isin(t)]
        if df_pair.empty:
            continue
        
        slope = 1E3 * (df_pair.iloc[0].RT - df_pair.iloc[1].RT)/(df_pair.iloc[0].Mass - df_pair.iloc[1].Mass)
        if slope > 0.:
#         if lo < slope < hi:
#         if any([i[0] < slope < i[1] for i in ranges]):
#             slopes.append(slope)
            plt.plot(df_pair.Mass, df_pair.RT, 'green')
            count += 1
        else:
            count_ignore += 1
        
        if not annotate:
            continue
            
        idmax = df_pair['Mass'].idxmax()
        plt.annotate(s=t[2], size=15, xy=(df_pair.loc[idmax].Mass, df_pair.loc[idmax].RT), 
                     textcoords="offset points", xytext=(-10, 10), ha='center', color='black')
        
        mass = '{:.2f}'.format(df_pair.loc[idmax].Mass)
        plt.annotate(s=mass, size=13, xy=(df_pair.loc[idmax].Mass, df_pair.loc[idmax].RT), 
                     textcoords="offset points", xytext=(10, -20), ha='center')

#     print('median {:.2f} lo-hi {:.3f}-{:.3f} accept {} reject {}'.format((lo+hi)/2, lo, hi, count, count_ignore))
    print(slopes)
    if not endpoints.empty:
        plt.scatter(endpoints.Mass, endpoints.RT, color='r')
        print(endpoints[['Mass', 'RT', 'Vol']])
    return plt, fig

In [ ]:
win_pos = 6200
win_size = 600

In [ ]:
df_win = df[(df.Mass > win_pos) & (df.Mass < win_pos + win_size)]
df_win.shape

In [ ]:
plt = plot_zones(df[df.Mass < 20000], df_win)
# plt.savefig('/Users/bryan/Downloads/tRNA_win.png', transparent=True)

In [ ]:
plt = plot_zone(df_win, trend=True)
# plt.savefig('/Users/bryan/Downloads/tRNA_win_dots.png', transparent=True)

In [ ]:
slope, _ = np.polyfit(df_win.Mass, df_win.RT, 1)
slope = slope * 1E3
print(slope)

In [ ]:
bcr_5p = base_calling_random(df_win, silence=True)
# plt, _ = plot_basecalling(*bcr_5p, annotate=False)
plt, _ = plot_basecalling2(*bcr_5p, annotate=False, median=slope)
plt.scatter(df_win.Mass, df_win.RT, color='C0')
plt.xticks(list(range(6200, 6800, 200)))
plt.grid(axis='x', which='major')
# plt.savefig('/Users/bryan/Downloads/tRNA_win_basecalling.png', transparent=True)
# plt.savefig('/Users/bryan/Downloads/tRNA_win_basecalling_select.png', transparent=True)

In [ ]:
df_

# Sliding

In [ ]:
win_pos = 1000
win_size = 600
win_step = 100
pairs = list()
while win_pos + win_size < 25000:
    df_win = df[(df.Mass > win_pos) & (df.Mass < win_pos + win_size)]
#     print(df_win.shape)
    if df_win.shape[0] > 1:
#         continue
        print('win {}-{}'.format(win_pos, win_pos+win_size))
        bcr_5p = base_calling_random(df_win, silence=True)
        accepted_pairs = process_slopes(*bcr_5p, radius=0.20)
        if accepted_pairs:
            pairs.extend(accepted_pairs)
    win_pos += win_step

In [ ]:
len(pairs)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 12))
plt.xlabel('Monoisotopic Mass (Da)', fontname="Arial", fontsize=15, color='black')
plt.ylabel('Retention Time (min)', fontname="Arial", fontsize=15, color='black')
plt.xticks(fontname="Arial", size=13, color='black')
plt.yticks(fontname="Arial", size=13, color='black')
# plt.scatter(df[df.Mass < 20000].Mass, df[df.Mass < 20000].RT)
for pair in pairs:
    if pair.iloc[0].Mass > 20000:
        continue
    plt.plot(pair.Mass, pair.RT)

# plt.savefig('/Users/bryan/Downloads/tRNA_sliding_wins.png', transparent=True)
plt.savefig('/Users/bryan/Downloads/tRNA_sliding_wins_lines_only.png', transparent=True)